In [1]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')

if not groq_api_key:
    raise ValueError("Groq API key not found in .env file")

In [2]:
# imports for precessing text
import os
import pickle
import re

# imports for creating pipeline for rag
from langchain.embeddings import HuggingFaceEmbeddings      # for embeddings
from langchain.vectorstores import Chroma                  # for vector store
from langchain.document_loaders import TextLoader        # for loading text
from langchain.text_splitter import RecursiveCharacterTextSplitter  # acting as base class for splitting text 
from langchain_groq import ChatGroq                         # for initializing LLM from groq
from langchain.schema import Document                   # converting simple text to document object

# For compressing the context of retrieved documents
from langchain.retrievers import ContextualCompressionRetriever     # for compressing retrieved documents context
from langchain.retrievers.document_compressors import LLMChainExtractor     # used in compression

# For creating the retrieval chain for chat history
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# For creating history aware retriever
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage     # for messages in history aware retriever


# Extra for now
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

c:\Users\zubay\.conda\envs\fyp\Lib\importlib\__init__.py:90: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)


In [3]:
# Initialize the HuggingFace Embedding Model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\zubay\AppData\Local\Temp\ipykernel_15388\1594320778.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\zubay\.conda\envs\fyp\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\zubay\.conda\envs\fyp\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be se

In [4]:
def load_documents(file_path):
    loader = TextLoader('resources/9thComputerScience_cleaned.txt')
    documents = loader.load()
    return documents

In [5]:
# Custom RecursiveCharacterTextSplitter with regex patterns for subtopics and chapters
class CustomTextSplitter(RecursiveCharacterTextSplitter):
    def __init__(self, **kwargs):
        subtopic_pattern = re.compile(r'(\d+(\.\d+)+)')
        chapter_separator = 'chapter end -------------------------------------'

        # Initialize with any other parameters, and add your separators
        super().__init__(separators=[chapter_separator], **kwargs)
        self.subtopic_pattern = subtopic_pattern

    def split_text(self, text):
        # First, split by chapters
        texts = super().split_text(text)
        documents = []
        
        # For each chapter, split by subtopic using the subtopic regex
        chapter_number = 1
        for chapter in texts:
            subtopic_splits = self._split_by_subtopic(chapter, chapter_number)
            documents.extend(subtopic_splits)
            chapter_number += 1
        
        return documents

    def _split_by_subtopic(self, text, chapter_number):
        # Use the subtopic regex to split text
        matches = list(self.subtopic_pattern.finditer(text))
        if not matches:
            # No subtopics found, return the full text as a single Document
            return [Document(page_content=text.strip(), metadata={"chapter": chapter_number})]
        
        subtopics = []
        start_idx = 0
        subtopic_number = 1
        
        for match in matches:
            end_idx = match.start()
            if start_idx != end_idx:
                subtopics.append(Document(
                    page_content=text[start_idx:end_idx].strip(),
                    metadata={"chapter": chapter_number, "subtopic": subtopic_number}
                ))
            start_idx = end_idx
            subtopic_number += 1
            
        # Append the remaining part as a subtopic
        subtopics.append(Document(
            page_content=text[start_idx:].strip(),
            metadata={"chapter": chapter_number, "subtopic": subtopic_number}
        ))
        
        return subtopics


In [6]:
# Create embeddings and handle storage
def embed_documents(split_docs, embedding_model):
    EMBEDDINGS_FOLDER = "embeddings"
    EMBEDDINGS_FILE = os.path.join(EMBEDDINGS_FOLDER, "emb01.pkl")

    if not os.path.exists(EMBEDDINGS_FOLDER):
        os.makedirs(EMBEDDINGS_FOLDER)

    if os.path.exists(EMBEDDINGS_FILE):
        print(f"Loading existing embeddings from {EMBEDDINGS_FILE}...")
        with open(EMBEDDINGS_FILE, 'rb') as f:
            embedded_docs = pickle.load(f)
            print("Embeddings loaded successfully.")
    else:
        print("Creating new embeddings...")
        texts = [doc.page_content for doc in split_docs]
        embedded_docs = embedding_model.embed_documents(texts)

        with open(EMBEDDINGS_FILE, 'wb') as f:
            pickle.dump(embedded_docs, f)
            print(f"Embeddings saved to {EMBEDDINGS_FILE}")

    return embedded_docs


In [7]:
# Store embeddings in Chroma vector store
def store_embeddings(split_docs, embedding_model):
    vector_store = Chroma.from_documents(split_docs, embedding_model) 
    return vector_store

In [8]:
def getting_retriever(llm,vector_store):
    """Opiton 01: Creating ContextualCompressionRetriever
    Contextual Compression will find the relevant records and only contains the relevant data from chunks instead of whole chunks
    Maximum Marginal Relevance (mmr) is used to get diverse set of documents.
    Option 02: SelfQueryRetrieval for filtering based on sources"""
    # Option 01
    compressor = LLMChainExtractor.from_llm(llm)
    compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vector_store.as_retriever(search_type = "mmr")
    )
    return compression_retriever

In [9]:
# Initialize the LLM
def initialize_llm(model_name="llama-3.1-70b-versatile", temperature=0.1):
    llm = ChatGroq(
        model= model_name,
        temperature=temperature,
    )
    return llm

In [10]:

# Load your document
documents = load_documents('resources/9thComputerScience_cleaned.txt')

# Split the text into smaller chunks
text_splitter = CustomTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = text_splitter.split_text(documents[0].page_content)
# print(split_docs[10])

# Create embeddings and store them
embedded_docs = embed_documents(split_docs, embedding_model)
vector_store = store_embeddings(split_docs, embedding_model)
llm = initialize_llm()

# Create the retriever having contextual compression
retriever = getting_retriever(llm,vector_store)


Loading existing embeddings from embeddings\emb01.pkl...
Embeddings loaded successfully.


In [11]:
# Just for testing the normal chain without chat history
system_prompt = (
    "Use the following pieces of retrieved context to answer "
    "the question. If foudn relevant to user query then do use it otherwise give it  "
    "from your own knowledge."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [12]:
result = rag_chain.invoke({"input": "can you give me an interesting topic from 1st chapter"})
result

{'input': 'can you give me an interesting topic from 1st chapter',
 'context': [Document(metadata={'chapter': 1, 'subtopic': 6}, page_content='1.1.3 Planning a Solution \nAfter analyzing a problem, we formulate a plan that may lead us towards the solution of a problem. This phase includes finding the right strategy for problem solving. Some of the strategies are:\ne Divide and Conquer: This strategy divides a complex problem into\nsmaller problems. Figure 1-3 Planning for success\ne Guess, Check and Improve: The designer guesses a solution to a problem and then checks the correctness of the solution. If the solution is not according to expectations, then he/she refines the solution. The refinement is an iterative process.\ne Act it Out: In this strategy the designer defines the list of “to-do” tasks. Afterwards he/she performs the task.\ne Prototype (Draw): This technique draws a pictorial representation of the solution. It is not the final solution. However, it may help a designer to 

In [13]:
result = rag_chain.invoke({"input": "guide me in each step further"})
result

{'input': 'guide me in each step further',
 'context': [Document(metadata={'chapter': 1, 'subtopic': 2}, page_content='1.1 Problem Solving Steps\nIn order to solve a problem, it is important to follow a systematic approach. In the following we discuss different steps that we can follow to solve a problem systematically.')],
 'answer': "Based on the problem-solving steps, I'll guide you through each step in more detail. Here's a general framework for systematic problem-solving:\n\n**Step 1: Define the Problem**\n\n* Clearly articulate the problem you're trying to solve. Be specific and concise.\n* Identify the key issues, symptoms, or challenges related to the problem.\n* Ask yourself questions like:\n\t+ What is the problem?\n\t+ Where is the problem occurring?\n\t+ When did the problem start?\n\t+ Who is affected by the problem?\n\t+ What are the consequences of not solving the problem?\n\n**Step 2: Gather Information**\n\n* Collect relevant data and facts about the problem.\n* Resear

## Implementing Chat history for context

In [14]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Then answer the user query "
    "precisely and ask follow-up related questions."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Creating a new retirever that is aware of the chat history. Rest of the things are same.
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [15]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

### Chat History Working

In [16]:
chat_history = []   # List of messages in the chat history

In [17]:
query = "can you give me an interesting topic from 1st chapter"
result = rag_chain.invoke({"input": query,"chat_history": chat_history})
# Append the user query and the system response to the chat history
chat_history.extend(
    [
        HumanMessage(content=query),
        AIMessage(content=result["answer"]),
    ]
)
print("Overall Result Object: ",result)
print("-----------------------------------")
print("Answer:",result["answer"])
print("-----------------------------------")
print("Context:",result["context"])
print("-----------------------------------")
print("Chat History:",chat_history)

Overall Result Object:  {'input': 'can you give me an interesting topic from 1st chapter', 'chat_history': [HumanMessage(content='can you give me an interesting topic from 1st chapter', additional_kwargs={}, response_metadata={}), AIMessage(content='From the 1st chapter, an interesting topic could be "Divide and Conquer" strategy. This strategy involves breaking down a complex problem into smaller, more manageable problems, making it easier to solve. It\'s a fascinating topic because it can be applied to various real-life situations, such as solving puzzles, managing large projects, or even tackling complex algorithms in computer science.', additional_kwargs={}, response_metadata={})], 'context': [Document(metadata={'chapter': 1, 'subtopic': 6}, page_content='1.1.3 Planning a Solution\nAfter analyzing a problem, we formulate a plan that may lead us towards the solution of a problem. This phase includes finding the right strategy for problem solving. Some of the strategies are:\ne Divid

In [18]:
# Follow up question without mentioning any particular term to test the memory
query = "will you give details about a particular poriton of it?"
result = rag_chain.invoke({"input": query,"chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=query),
        AIMessage(content=result["answer"]),
    ]
)
print("Overall Result Object: ",result)
print("-----------------------------------")
print("Answer:",result["answer"])
print("-----------------------------------")
print("Context:",result["context"])
print("-----------------------------------")
print("Chat History:",chat_history)

Overall Result Object:  {'input': 'will you give details about a particular poriton of it?', 'chat_history': [HumanMessage(content='can you give me an interesting topic from 1st chapter', additional_kwargs={}, response_metadata={}), AIMessage(content='From the 1st chapter, an interesting topic could be "Divide and Conquer" strategy. This strategy involves breaking down a complex problem into smaller, more manageable problems, making it easier to solve. It\'s a fascinating topic because it can be applied to various real-life situations, such as solving puzzles, managing large projects, or even tackling complex algorithms in computer science.', additional_kwargs={}, response_metadata={}), HumanMessage(content='will you give details about a particular poriton of it?', additional_kwargs={}, response_metadata={}), AIMessage(content='Let\'s dive deeper into the "Divide and Conquer" strategy. \n\nIn the Divide and Conquer approach, a problem is broken down into smaller sub-problems, and each 